In [1]:
import torch
from data import ClimateDataset
from models import ClassificationModel
from engine import Engine

In [2]:
model_checkpoint='gpt2'

In [3]:
results = {'acc':[],
           'f1':[],
           'wd':[]}

In [4]:
lr = 1.0e-4
wd = 0.01

print(f'Grid search {model_checkpoint}, lr: {lr}')
data = ClimateDataset(model_to_train=1,model_checkpoint=model_checkpoint,batch_size=32)
data.setup_dataloaders()
model = ClassificationModel(model_checkpoint=data.model_checkpoint,num_labels=data.num_labels)
trainer = Engine(epochs=10,labels=data.labels)
trainer.model = model.model
trainer.dataset_encoded = data.dataset_encoded
acc, f1 = trainer.run(lr=lr,
                        wd=wd,
                        train_dataloader=data.train_dataloader,
                        eval_dataloader=data.eval_dataloader,
                        test_dataloader=data.test_dataloader,
                        early_stop=3)
results['acc'].append(acc)
results['f1'].append(f1)
results['wd'].append(wd)
print('### '*10)
print(results)
print('### '*10)

Grid search gpt2, lr: 0.0001


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


1 / 10: Train Loss:	0.7142	Val Loss:	0.5473	Accuracy:	0.7396	F1:	0.4332
2 / 10: Train Loss:	0.5005	Val Loss:	0.4887	Accuracy:	0.7899	F1:	0.6327
3 / 10: Train Loss:	0.4572	Val Loss:	0.4670	Accuracy:	0.8009	F1:	0.6501
4 / 10: Train Loss:	0.4083	Val Loss:	0.4038	Accuracy:	0.8359	F1:	0.7537
5 / 10: Train Loss:	0.3212	Val Loss:	0.4607	Accuracy:	0.8184	F1:	0.6901
6 / 10: Train Loss:	0.2713	Val Loss:	0.4253	Accuracy:	0.8403	F1:	0.7873
7 / 10: Train Loss:	0.2036	Val Loss:	0.4406	Accuracy:	0.8490	F1:	0.7915
8 / 10: Train Loss:	0.1314	Val Loss:	0.7975	Accuracy:	0.8337	F1:	0.7371
9 / 10: Train Loss:	0.0683	Val Loss:	1.0197	Accuracy:	0.8206	F1:	0.7037
10 / 10: Train Loss:	0.0813	Val Loss:	0.8264	Accuracy:	0.8271	F1:	0.7232
No improvement for 3 epochs. Stopping early.
best (higgest macro f1-score) val results:
              precision    recall  f1-score   support

       bknow       0.84      0.41      0.55       119
      struct       0.82      0.97      0.89       338

    accuracy               